In [1]:
import requests
from bs4 import BeautifulSoup
import codecs
import time
import re
import pandas as pd
import numpy as np

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'
    }
# необзодимые url'ы (25 штук)
urls = (('https://rateyourmusic.com/customchart?page=%s '*25 % (tuple(map(str,range(1,26))))).split(' '))[:-1]

In [3]:
files_name = [('page_' + str(i) + '.html') for i in range (1,26)]

'''
# получаем html + записываем в файл
def get_html(url,file_name,h = headers):
    #отправляем get запрос
    r = requests.get(url,headers = h)
    #записываем html в файл
    with open(file_name, 'w') as data:
        data.write(r.text)
        
# в цикле сохраняем 40 html страниц
for i in range(39,40):
    get_html(urls[i],files_name[i])
    time.sleep(1)
'''

"\n# получаем html + записываем в файл\ndef get_html(url,file_name,h = headers):\n    #отправляем get запрос\n    r = requests.get(url,headers = h)\n    #записываем html в файл\n    with open(file_name, 'w') as data:\n        data.write(r.text)\n        \n# в цикле сохраняем 40 html страниц\nfor i in range(39,40):\n    get_html(urls[i],files_name[i])\n    time.sleep(1)\n"

In [132]:
# чистим sats
def get_stats(*stats):
    rym_rating = np.array([re.findall(r'\d\.\d+', stats[i][0])[0] for i in range(40)], dtype=float)
    
    rating1     = np.array([re.findall(r'[0-9,]+', stats[i][1])[0] for i in range(40)],dtype = str)
    rating     = np.array([float(re.sub(',','.',rate))*1000  if ',' in str(rate) else float(rate) for rate in rating1])

    review     = np.array([re.sub(',','',re.findall(r'[0-9,]+', stats[i][2])[0]) for i in range(40)],dtype=float) 
    return rym_rating,rating,review

# парсим
def data_mining(page_name):
    f = open(page_name, 'r')
    html = f.read()
    soup = BeautifulSoup(html, features ='lxml') 
    
    # content - контейнер с нашими данными
    content =  soup.find('table',{'class' : 'mbgen'})
    # в 'chart-detail' - название группы, альбома; 'chart-stats'  - статистика + мусор какой-то
    items  = content.find_all('div',{'class' : ['chart_detail', 'chart_stats']})
    
    # исполнитель
    singer = np.array([items[i].find('a',{'class' : 'artist'}).text for i in range(0,80,2)],dtype='str')
    album  = np.array([items[i].find('a',{'class' : 'album' }).text for i in range(0,80,2)],dtype='str')
    # рейтинг
    stats = [items[i].find('a').text.split('             ') for i in range(1,80,2)]
    # результат get_stats
    rym_rating, rating, review = np.array(get_stats(*stats)[0]), np.array(get_stats(*stats)[1]), np.array(get_stats(*stats)[2])
    # добавляем полngученные значения в словарь
    library['singer'].append(singer)
    library['album'].append(album)
    library['rym_rating'].append(rym_rating)
    library['rating'].append(rating)
    library['review'].append(review)
def all_pages():
    # парсим 25 страниц
    [data_mining(i) for i in files_name]
    # соединяем value
    for el in library:
        library[el] = np.concatenate(library[el])

In [136]:
library = {'singer': [], 'album' : [],'rym_rating' : [], 'rating' : [], 'review' : []}
all_pages()

In [138]:
data = pd.DataFrame(library)
# меняем порядок столбцов
data = data[['singer','album','rym_rating','rating','review',]]
# сохраняем наш DataFrame в data.csv
data.to_csv('data.csv', sep='\t', encoding='utf-8')

In [233]:
data.head(5)

,singer,album,rym_rating,rating,review
0,Radiohead,OK Computer,4.23,41673.0,2202.0
1,Pink Floyd,The Dark Side of the Moon,4.23,35458.0,1847.0
2,The Velvet Underground,The Velvet Underground & Nico,4.29,27824.0,1203.0
3,Radiohead,Kid A,4.19,34019.0,1304.0
4,Pink Floyd,Wish You Were Here,4.27,28795.0,1130.0


## TOП 1 по количеству упоминаний

In [216]:
top1 = data['singer'].value_counts().idxmax()
top1

'Miles Davis'

## ТОП 10 по количеству проголосовавших

In [239]:
# common = [[unique_singer1,common_rate1],[unique_singer2, common_rate2],...]
rate_and_singer = [[el,sum(data[data['singer'] == el]['rating'].tolist())] for el in set(data['singer'])]
# сделаем словарь и из него DataFrame
top2 = {'singer':[el[0] for el in rate_and_singer],
              'singer_rate':[el[1] for el in rate_and_singer]}
top2_df = pd.DataFrame(top_rating)
# отсортируем по singer_rate наш DataFrame top_df
top2_df = top_df.sort_values(by='singer_rate',ascending=False)
#print(top_df.loc['singer','singer_rate'].head(10))

In [240]:
top2 = top2_df.loc[:,['singer','singer_rate']].head(10)
top2

,singer,singer_rate
52,Radiohead,188430.0
487,The Beatles,182181.0
131,Pink Floyd,155550.0
409,David Bowie,114193.0
65,Led Zeppelin,104991.0
413,Bob Dylan,98562.0
287,Black Sabbath,86508.0
48,The Velvet Underground,67800.0
139,Metallica,67210.0
35,Miles Davis,65662.0


## ТОП 10 исполнителей с самой высокой средней оценкой

In [251]:
singer_and_rym = [[el, data[data['singer'] == el]['rym_rating'].mean()] for el in set(data['singer'])]
top3 = {'singer':[el[0] for el in singer_and_rym],
                   'average_rym':[el[1] for el in singer_and_rym]}
top3_df = pd.DataFrame(top3).sort_values(by='average_rym', ascending=False)

In [252]:
top3_df.head(10)

,average_rym,singer
477,4.350000,Mingus
394,4.210000,The Beach Boys
152,4.180000,Television
323,4.180000,Wu-Tang Clan
206,4.180000,Nas
155,4.130000,Godspeed You Black Emperor!
243,4.130000,Madvillain
353,4.123333,The Jimi Hendrix Experience
17,4.120000,Genius/GZA
178,4.120000,Joy Division
